In [0]:
# from google.colab import drive
# drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train_path = "training"
test_path = "test_set_images"

In [0]:
import numpy as np 
import os
import skimage.io as io
import tensorflow as tf
from keras.models import *
from keras.layers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, TensorBoard
from keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# https://stackoverflow.com/questions/43547402/how-to-calculate-f1-macro-in-keras

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
# foreground_threshold = 0.25

# def patch_to_label(patch):
#     df = np.mean(patch)
#     if df > foreground_threshold:
#         return 1
#     else:
#         return 0
      
# def get_label(y, patch_size=16):
#     label = []
#     y_array = K.eval(y[:,:,:,0])
#     for im in y_array:
#       for j in range(0, im.shape[1], patch_size):
#         for i in range(0, im.shape[0], patch_size):
#             patch = im[i:i + patch_size, j:j + patch_size]
#             label.append(patch_to_label(patch))
#     return np.array(label)
  
# def patch_f1(y_true, y_pred):
#     true_label = get_label(y_true)
#     pred_label = get_label(y_pred)
#     return f1(true_label, pred_label)

In [0]:
# # credits: https://www.kaggle.com/guglielmocamporese/macro-f1-score-keras

# def f1(y_true, y_pred):
#     #y_pred = K.round(y_pred)
#     #y_pred = K.cast(K.greater(K.clip(y_pred, 0, 1), THRESHOLD), K.floatx())
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return K.mean(f1)
  
# def f1_loss(y_true, y_pred):
    
#     tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
#     tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
#     fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
#     fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

#     p = tp / (tp + fp + K.epsilon())
#     r = tp / (tp + fn + K.epsilon())

#     f1 = 2*p*r / (p+r+K.epsilon())
#     f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
#     return 1 - K.mean(f1)

In [0]:
def conv2d_block(input_tensor, n_filter, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(n_filter, kernel_size=kernel_size, kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(n_filter, kernel_size=kernel_size, kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [0]:
def unet(pretrained_weights = None,
         input_size = (None,None,3),
         n_filter=16,
         dropout=True, dropout_rate=0.5,
         batchnorm=True
        ):
  
    inputs = Input(input_size)
    
    conv1 = conv2d_block(inputs, n_filter, kernel_size=3, batchnorm=batchnorm)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = conv2d_block(pool1, n_filter*2, kernel_size=3, batchnorm=batchnorm)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = conv2d_block(pool2, n_filter*4, kernel_size=3, batchnorm=batchnorm)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = conv2d_block(pool3, n_filter*8, kernel_size=3, batchnorm=batchnorm)
    conv4 = Dropout(dropout_rate)(conv4) if dropout else conv4
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = conv2d_block(pool4, n_filter*16, kernel_size=3, batchnorm=batchnorm)
    conv5 = Dropout(dropout_rate)(conv5) if dropout else conv5

    up6 = UpSampling2D(size = (2, 2))(conv5)
    up6 = conv2d_block(up6, n_filter*8, kernel_size=2, batchnorm=batchnorm)
    merge6 = concatenate([conv4, up6], axis = 3)
    conv6 = conv2d_block(merge6, n_filter*8, kernel_size=3, batchnorm=batchnorm)
    
    up7 = UpSampling2D(size = (2, 2))(conv6)
    up7 = conv2d_block(up7, n_filter*4, kernel_size=2, batchnorm=batchnorm)
    merge7 = concatenate([conv3, up7], axis = 3)
    conv7 = conv2d_block(merge7, n_filter*4, kernel_size=3, batchnorm=batchnorm)
    
    up8 = UpSampling2D(size = (2, 2))(conv7)
    up8 = conv2d_block(up8, n_filter*2, kernel_size=2, batchnorm=batchnorm)
    merge8 = concatenate([conv2, up8], axis = 3)
    conv8 = conv2d_block(merge8, n_filter*2, kernel_size=3, batchnorm=batchnorm)
    
    up9 = UpSampling2D(size = (2, 2))(conv8)
    up9 = conv2d_block(up9, n_filter, kernel_size=2, batchnorm=batchnorm)
    merge9 = concatenate([conv1, up9], axis = 3)
    conv9 = conv2d_block(merge9, n_filter, kernel_size=3, batchnorm=batchnorm)
    
#     conv10 = Conv2D(2, 1, kernel_initializer="he_normal", padding="same")(conv9)
#     if batchnorm:
#         conv10 = BatchNormalization()(conv10)
#     conv10 = Activation("relu")(conv10)
    
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs = inputs, outputs = conv10)

    model.compile(optimizer = Adam(), loss = 'binary_crossentropy', metrics = [f1, 'accuracy'])
    
    if(pretrained_weights):
        model.load_weights(filepath=pretrained_weights)

    return model

In [0]:
# def adjustImage(img, mask):
#     img = img/255
#     mask = mask/255
#     mask[mask > 0.5] = 1
#     mask[mask <= 0.5] = 0
#     return img, mask

In [0]:
def preprocess_mask(mask):
    mask = mask/255
    mask[mask > 0.5] = 1
    mask[mask <= 0.5] = 0
    return mask
  
def preprocess_img(img):
    return img/255

In [0]:
def trainvalGenerator(batch_size, train_path, image_folder, mask_folder,
                      aug_dict, train_dir = None, val_dir = None,
                      target_size = (400,400), seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_dict = aug_dict.copy()
    image_dict["preprocessing_function"] = preprocess_img
    image_datagen = ImageDataGenerator(**image_dict)
    
    mask_dict = aug_dict.copy()
    mask_dict["preprocessing_function"] = preprocess_mask
    mask_datagen = ImageDataGenerator(**mask_dict)
    
    image_generator_train = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = "rgb",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = train_dir,
        save_prefix  = "image",
        seed = seed,
        subset = "training")
    
    image_generator_val = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = "rgb",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = val_dir,
        save_prefix  = "image",
        seed = seed+1,
        subset = "validation")
    
    mask_generator_train = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = "grayscale",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = train_dir,
        save_prefix  = "mask",
        seed = seed,
        subset = "training")
    
    mask_generator_val = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = "grayscale",
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = val_dir,
        save_prefix  = "mask",
        seed = seed+1,
        subset = "validation")
    
    return zip(image_generator_train, mask_generator_train), zip(image_generator_val, mask_generator_val)

In [0]:
# argument for data augmentation
data_gen_args = dict(rotation_range=90,
                     width_shift_range=0.1,
                     height_shift_range=0.1,
                     horizontal_flip=True,
                     vertical_flip=True,
                     fill_mode='reflect',
                     validation_split=0)

In [0]:
!rm -rf train_image
!rm -rf val_image
!mkdir train_image
!mkdir val_image

In [23]:
trainGen, valGen = trainvalGenerator(batch_size=2, 
                                     train_path=train_path, 
                                     image_folder='images', mask_folder='groundtruth',
                                     aug_dict=data_gen_args, 
                                     train_dir = "train_image",
                                     val_dir = "val_image",
                                     target_size = (400, 400), seed = 1)


Found 100 images belonging to 1 classes.
Found 0 images belonging to 1 classes.
Found 100 images belonging to 1 classes.
Found 0 images belonging to 1 classes.


In [0]:
model = unet(n_filter=32, dropout=True)
# model.summary()

In [0]:
callbacks = [
#     EarlyStopping(patience=10, verbose=1),
#     ModelCheckpoint('weights_no_val.h5', monitor='val_loss', save_best_only=True, verbose=1),
    EarlyStopping(monitor='loss', patience=10, verbose=1),
    ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=1, min_lr=1e-5),
    ModelCheckpoint('weights_no_val_32_10layers_drop.h5', monitor='loss', save_best_only=True, verbose=1),
    TensorBoard(log_dir='tensorboard/', write_graph=True, write_images=True)
]

In [41]:
print('*'*30)
print('Fitting model...')
print('*'*30)
history = model.fit_generator(generator=trainGen, steps_per_epoch=50,
#                               validation_data=valGen, validation_steps=50,
                              epochs=500, callbacks=callbacks)

******************************
Fitting model...
******************************
Epoch 1/500
50/50 [==============================] - 31s 621ms/step - loss: 0.4969 - f1: 0.3242 - acc: 0.7596

Epoch 00001: loss improved from inf to 0.49692, saving model to weights_no_val_32_10layers_drop.h5
Epoch 2/500
50/50 [==============================] - 24s 487ms/step - loss: 0.4038 - f1: 0.4060 - acc: 0.8157

Epoch 00002: loss improved from 0.49692 to 0.40382, saving model to weights_no_val_32_10layers_drop.h5
Epoch 3/500
50/50 [==============================] - 24s 484ms/step - loss: 0.3524 - f1: 0.5211 - acc: 0.8375

Epoch 00003: loss improved from 0.40382 to 0.35243, saving model to weights_no_val_32_10layers_drop.h5
Epoch 4/500
50/50 [==============================] - 24s 484ms/step - loss: 0.3623 - f1: 0.5671 - acc: 0.8377

Epoch 00004: loss did not improve from 0.35243
Epoch 5/500
50/50 [==============================] - 24s 484ms/step - loss: 0.3459 - f1: 0.5113 - acc: 0.8381

Epoch 00005: l

In [0]:
import skimage.io as io
def testGenerator(test_path, num_image = 50):
    for i in range(1,num_image+1):
        img = io.imread(os.path.join(test_path, "test_%d"%i, "test_%d.png"%i))
        img = img / 255
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [0]:
testGene = testGenerator(test_path)
# model_predict = unet(pretrained_weights = "weights_no_val_32.h5", input_size = (608,608,3))

In [0]:
# model = unet(n_filter=32)
# model.load_weights(filepath="weights_no_val_32.h5")

In [45]:
result = model.predict_generator(testGene, 50, verbose=1)

50/50 [==============================] - 12s 242ms/step


In [49]:
def saveResult(save_path, npyfile):
    for i,item in enumerate(npyfile):
        img = item[:,:,0]
        io.imsave(os.path.join(save_path, "%d_predict.png"%(i+1)), img)

!mkdir test        
saveResult("test", result)

/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:122: UserWarning: Possible precision loss when converting from float32 to uint16
  .format(dtypeobj_in, dtypeobj_out))


In [0]:
import os
import numpy as np
import matplotlib.image as mpimg
import re

foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

# assign a label to a patch
def patch_to_label(patch):
    df = np.mean(patch)
    if df > foreground_threshold:
        return 1
    else:
        return 0


def mask_to_submission_strings(image_filename):
    """Reads a single image and outputs the strings that should go into the submission file"""
    img_number = int(re.search(r"\d+", image_filename).group(0))
    im = mpimg.imread(image_filename)
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number, j, i, label))


def masks_to_submission(submission_filename, *image_filenames):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for fn in image_filenames[0:]:
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn))

In [0]:
submission_filename = 'submission.csv'
image_filenames = []
predict_path = 'test/'
for i in range(1, 51):
    image_filename = predict_path + '%d' % i + '_predict.png'
#     print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(submission_filename, *image_filenames)